In [1]:
from google.colab import files
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Bot_sides')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

In [4]:
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
import os, sys
import re
import string
import itertools

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

Importing libraries for conversation model

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
convo_model = load_model('chatbot_model.h5')
import json
import random
responses = json.loads(open('response.json').read())
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Importing Functions for the conversation model

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res)]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

Importing functions for running conversation

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return tag,result

def chatbot_response(text):
    ints = predict_class(text, convo_model)
    l = len(ints)
    index = 0
    new_tag, res = getResponse(ints, intents)
    return new_tag,res

In [11]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [12]:
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [13]:
def text_clean(text):
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove urls
    text = re.sub(r'<([^>]*)>', ' ', text) # remove emojis
    text = re.sub(r'@\w+', ' ', text) # remove at mentions
    text = re.sub(r'#', '', text) # remove hashtag symbol
    text = re.sub(r'[0-9]+', ' ', text) # remove numbers
    text = replace_contractions(text)
    pattern = re.compile(r"[ \n\t]+")
    text = pattern.sub(" ", text)      
    text = "".join("".join(s)[:2] for _, s in itertools.groupby(text))    
    text = re.sub(r'[^A-Za-z0-9,?.!]+', ' ', text) # remove all symbols and punctuation except for . , ! and ?
    return text.strip()

In [14]:
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(s): return [w.text.lower() for w in nlp(text_clean(s))]

In [15]:
TEXT = Field(sequential=True, tokenize=tokenizer, include_lengths=True, use_vocab=True)
TARGET = Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None, is_target =False)

In [16]:
data_fields = [
    (None, None),
    ("message", TEXT), 
    ("label", TARGET)
]


In [17]:
vec = torchtext.vocab.Vectors('glove.6B.100d.txt')

In [18]:
train_data, val_data, test_data = TabularDataset.splits(path='./', format='csv', train='train.csv', validation='val.csv', test='test.csv', fields=data_fields, skip_header=True)

In [19]:
MAX_VOCAB_SIZE = 100_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE,
                 vectors=vec)

TARGET.build_vocab(train_data)

In [20]:
train_data.fields

{None: None,
 'label': <torchtext.data.field.Field at 0x7fa5043915f8>,
 'message': <torchtext.data.field.Field at 0x7fa504391588>}

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
def read_vocab(path):
    #read vocabulary pkl 
    import pickle
    pkl_file = open(path, 'rb')
    vocab = pickle.load(pkl_file)
    pkl_file.close()
    return vocab

In [23]:
vocab_path = 'vocab.pkl'
vocab = read_vocab(vocab_path)

In [24]:
vocab_size = len(TEXT.vocab)
embedding_dim = 100
n_hidden = 64
n_out = 2

In [25]:
class ConcatPoolingGRUAdaptive(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_hidden, n_out, pretrained_vec, dropout, bidirectional=True):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.bidirectional = bidirectional
        
        self.emb = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.emb.weight.data.copy_(pretrained_vec)
        self.emb.weight.requires_grad = False
        self.gru = nn.GRU(self.embedding_dim, self.n_hidden, bidirectional=bidirectional)
        if bidirectional:
            self.fc = nn.Linear(self.n_hidden*2*2, self.n_out)
        else:
            self.fc = nn.Linear(self.n_hidden*2, self.n_out)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, seq, lengths):
        bs = seq.size(1)
        self.h = self.init_hidden(bs)
        seq = seq.transpose(0,1)
        embs = self.emb(seq)
        embs = embs.transpose(0,1)
        embs = pack_padded_sequence(embs, lengths)
        gru_out, self.h = self.gru(embs, self.h)
        gru_out, lengths = pad_packed_sequence(gru_out)        
        
        avg_pool = F.adaptive_avg_pool1d(gru_out.permute(1,2,0),1).view(bs,-1)
        max_pool = F.adaptive_max_pool1d(gru_out.permute(1,2,0),1).view(bs,-1) 
        
        cat = self.dropout(torch.cat([avg_pool,max_pool],dim=1))
        
        outp = self.fc(cat)
        return F.log_softmax(outp)
    
    def init_hidden(self, batch_size): 
        if self.bidirectional:
            return torch.zeros((2,batch_size,self.n_hidden)).to(device)
        else:
            return torch.zeros((1,batch_size,self.n_hidden)).cuda().to(device)

In [26]:
model = ConcatPoolingGRUAdaptive(vocab_size, embedding_dim, n_hidden, n_out, train_data.fields['message'].vocab.vectors, 0.5).to(device)

In [27]:
model.load_state_dict(torch.load('tut4-model.pt'))

<All keys matched successfully>

In [28]:
import numpy as np

def predict(model, vocab, sentence):
    tokenized = [w.text.lower() for w in nlp(text_clean(sentence))]  # tokenize the sentence
    indexed = [vocab.stoi[t] for t in tokenized]         # convert to integer sequence
    length = [len(indexed)]                             # compute no. of words
    tensor = torch.LongTensor(indexed).to(device)                                       # convert to tensor
    tensor = torch.reshape(tensor, (length[0], 1))                     # reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)
    prediction = model(tensor, length_tensor) 
    n_prediction = prediction.detach().cpu().numpy() # it is 1 with 
    numerator = np.exp(n_prediction[0,0])+np.exp(n_prediction[0,1])
    zero_score = np.exp(n_prediction[0,0])/numerator
    one_score = np.exp(n_prediction[0,1])/numerator
    if zero_score>one_score:
        return 'Non Depressive',zero_score
    else:
        return 'Depressive',one_score
   
    #print(torch.max(prediction,1))
    #pred_idx = torch.max(prediction, 1)[1]
    #if pred_idx == torch.tensor([0]).to(device):
    #    print('Non Depressive')
    #else:
    #    print('Depressive')

In [29]:
def extracting_sentence(para):
    tokens = nltk.sent_tokenize(para)
    return tokens

In [30]:
predict(model, vocab, 'Sorry for everything')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


('Depressive', 0.85128194)

In [59]:
predict(model, vocab, 'I have not done anything useful today' )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


('Depressive', 0.5926968)

In [31]:
def run(sentence,model,vocab):
    count = 0 
    for i in range(5):
        s_type,percentage = predict(model,vocab,sentence)
        if s_type== 'Depressive':
            count+=1
        else:
            count-=1
    if count >=0:
        return "Depressive"
    else:
        return "Non Depressive"

In [32]:
def model_prediction(model,vocab,para):
    sentence_list = extracting_sentence(para)
    l = len(sentence_list)
    depression_score = 0
    count = 0 
    for sentence in sentence_list:
        m = run(sentence, model, vocab) 
        if m == 'Depressive':
            depression_score+=1            
        count+=1
    if count%2==0:
        if depression_score>=count//2:
            return 'Depressive'
        else:
            return 'Non Depressive'
    else:
        if depression_score>=count//2+1:
            return 'Depressive'
        else:
            return 'Non Depressive'

In [33]:
def update_discrete_score(count,messege,depression_score):
    if messege == 'Depressive':
        depression_score+=1
    count+=1
    return depression_score,count

In [34]:
import random

In [35]:
def breathe_response():
    inp = input("You: ")
    if inp.lower()=='yes':
        print("Maya:{}".format(responses['questions'][8]['breathing_response']['yes']))
    print('Maya:{}'.format(responses['questions'][11]['final response'][0]))


In [36]:
def story_response():
    inp = input("You: ")
    if inp.lower()=='yes':
        print('Maya:{}'.format(responses['questions'][6]['story_response']['yes']))
        inp = input('You: ')
        if inp.lower()=='yes':
            print('Maya:{}'.format(responses['questions'][7]['kindness_story'][0]))
    print('Maya:{}'.format(responses['questions'][6]['story_response']['no']))
    breathe_response()

In [37]:
from random import randrange


Among the 8 main questions, if 0-2 questions sound depressed then no depression, 3-4 mild depression, 5-8 severe depression

In [38]:
def high_depression_response():
    l = len(responses['questions'][3]['depression_response'])
    i = randrange(l)
    rand_dic = random.choice(responses['questions'][3]['depression_response'])
    for question in responses['questions'][3]['depression_response'][i]:
        print('Maya:{}'.format(question))
        inp = input("You: ")
        if inp.lower()=='yes':
            print('Maya:{}'.format(responses['questions'][3]['depression_response'][i][question]['yes']))
        else:
            print('Maya:{}'.format(responses['questions'][3]['depression_response'][i][question]['no']))
    story_response()
            

In [39]:
def mild_depression_response():
    rand_elt = random.choice(responses['questions'][10]['mild depression response'])
    print('Maya:{}'.format(rand_elt))
    story_response()

In [40]:
def no_depression_response():
    rand_elt = random.choice(responses['questions'][9]['no_depression_response'])
    print('Maya:{}'.format(rand_elt))
    story_response()


In [42]:
import random

In [47]:
def open_ended_question():
    topic_list = responses['questions'][13]['open_prompts']
    inds = random.sample(range(5),2) #topics are at topic_list[inds[0]], topic_list[inds[1]]
    print('Maya:{}'.format(responses['questions'][4]['open_ended_questions'][topic_list[inds[0]]]))
    inp = input("You: ")
    print('Maya:{}'.format(responses['questions'][5]['open_ended_response'][topic_list[inds[0]]]))
    print('{}'.format(responses['questions'][4]['open_ended_questions'][topic_list[inds[1]]]))
    inp = input("You: ")
    print('Maya:{}'.format(responses['questions'][5]['open_ended_response'][topic_list[inds[1]]]))

In [44]:
def depression_dealer(d_type):
    if d_type in {'mild','high'}:
        for question in responses['questions'][2]['optional_questions']:
            print('Maya:{}'.format(question))
            inp = input("You: ")
            if inp.lower() == 'yes':
                print('Maya:{}'.format(responses['questions'][2]['optional_questions'][question]['yes']))
            else:
                print('Maya:{}'.format(responses['questions'][2]['optional_questions'][question]['no']))
            next_inp = input("You: ")
            if next_inp.lower()=='yes':
                open_ended_question()
            if d_type=='mild':
                mild_depression_response()
            else:
                high_depression_response()
    else:
        print('Maya: Would you be down to talk about some relaxing stuff, or may be things which will help you do some self reflection and makes you feel better?(yes/no)')
        inp = input("You: ")
        if inp.lower()=='yes':
            open_ended_question()
        no_depression_response()

This is another chat function where the calculation for depression is a little different from the above one :)

In [48]:
def chat():
    print("Start talking with the bot!")
    flag = False
    depression_score = 0
    count = 0
    while True:
        inp = input("You: ")
        if inp.lower() == 'result':
            if 6<=depression_score<=10:
                print('It does not sound like things are going alright with you and you should see a doctor for helping you further')
                depression_response()
            elif 3<=depression_score<=5:
                mild_depression_response()
            else:
                no_depression_response()
            break
        if inp.lower() == 'start':
            flag = True
            ques_list = responses['questions'][0]['q_wo_followup'].copy()
            random.shuffle(ques_list)
            for question in ques_list:
                print('Maya:{}'.format(question))
                inp = input("You: ")
                messege = model_prediction(model,vocab,inp)
                depression_score,count = update_discrete_score(count,messege,depression_score)
            for question in responses['questions'][1]['q_w_followup']:
                print('Maya:{}'.format(question))
                inp = input("You: ")
                if inp.lower() == 'yes':
                    print('Maya:{}'.format(responses['questions'][1]['q_w_followup'][question]['yes']))
                else:
                    print('Maya:{}'.format(responses['questions'][1]['q_w_followup'][question]['no']))
                if question not in responses['questions'][12]['followup_pos'] or inp.lower()!=responses['questions'][12]['followup_pos'][question]:
                    next_inp = input("You: ")
                    messege = model_prediction(model, vocab, next_inp)
                    depression_score,count = update_discrete_score(count,messege,depression_score)
                else:
                    count+=1
#count depression score, if you are in a no range, do something, if you are in a mild range, do something, 
#if you are very depressed range, do something. 
            print(depression_score)
            if 6<=depression_score<=8: # high depression range
                depression_dealer('high')
            elif 4<=depression_score<=5:
                depression_dealer('mild')
            else:
                depression_dealer('no')
            break
        if not flag:
            new_tag,result = chatbot_response(inp)
            print('Maya:{}'.format(result))

chat()

Start talking with the bot!
You: hi
Maya:Hello, nice to see you!!!
You: start
Maya:How do you feel about yourself?
You: good


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Maya:How is your energy level today?
You: good
Maya:How do spend time during the day? What about night, before you go to sleep?
You: read 
Maya:How have you feeling the past few days?
You: good
Maya:Have you talked recently with any family or friends? (Yes/No)
You: no
Maya:Umm I see. How is your relationship with them?
You: good
Maya:Are you sleeping well recently? (Yes/No)
You: yes
Maya:Wow having good sleep feels good.
Maya:Are you worrying over anything recently? (Yes/No)
You: no
Maya:Wow that is good to know
Maya:Are you feeling okay with performing routine tasks?(Yes/No)
You: yes
Maya:That's good to know that you are doing well because sometimes routine tasks appear tiresome to some people
0
Maya: Would you be down to talk about some relaxing stuff, or may be things which will help you do some self reflection and makes you feel better?(yes/no)
You: yes
Maya:Have you done or made something silly recently and still everytime you remember it, it fills up your mind with joy and laught